Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE: MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework: MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

MindSPONGE tutorial protein 05: Homework answers.

In [1]:
from mindspore import context
import sys
sys.path.insert(0, '../../src')
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units, WithEnergyCell
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.control import VelocityVerlet
from sponge.callback import WriteH5MD, RunInfo
from sponge.control import Langevin
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion
from sponge.sampling import Metadynamics
from sponge.function import PI
from sponge.system import Protein

[WARNING] ME(21834:139748229682368,MainProcess):2023-08-15-16:38:09.213.220 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21834:139748229682368,MainProcess):2023-08-15-16:38:09.216.082 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21834:139748229682368,MainProcess):2023-08-15-16:38:09.217.235 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21834:139748229682368,MainProcess):2023-08-15-16:38:09.218.031 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21834:139748229682368,MainProcess):2023-08-15-16:38:09.218.904 [mindspore/common/api.py:897] 'mindspore.ms_function

In [2]:
context.set_context(mode=context.GRAPH_MODE, device_target="Ascend")
set_global_units('nm', 'kj/mol')

Step 1: build the system

In [3]:
pdb_name = '../pdb/case1.pdb'
out_pdb = '../case1_sol.pdb'
mol = Protein(pdb_name, template=['protein0.yaml'], rebuild_hydrogen=True)
mol.fill_water(edge=0.4, pdb_out=out_pdb, template='water.spce.yaml')

[MindSPONGE] Adding 57 hydrogen atoms for the protein molecule in 0.006 seconds.
[MindSPONGE] The box size used when filling water is: [2.1184928 2.285693  1.549593 ]
[MindSPONGE] The edge gap along x axis is 0.4.
[MindSPONGE] The edge gap along y axis is 0.3999999523162842.
[MindSPONGE] The edge gap along z axis is 0.4.
[MindSPONGE] Totally 172 waters is added to the system!


/tmp/ipykernel_21834/3903672113.py:4: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  mol.fill_water(edge=0.4, pdb_out=out_pdb, template='water.spce.yaml')


[MindSPONGE] Adding water molecule task finished!


_MoleculeFromPDB<>

Step 2: Minimization

In [4]:
energy = ForceField(mol, parameters=['AMBER.FF14SB', 'SPCE'], use_pme=False)
min_opt = SteepestDescent(mol.trainable_params(), 1e-6)
sim = WithEnergyCell(mol, energy)
md = Sponge(sim, optimizer=min_opt)
run_info = RunInfo(10)
cb_h5md = WriteH5MD(mol, '../p05_1.h5md', save_freq=10, save_last_pdb='../p05.pdb', write_image=False)
md.run(500, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:38:18
[MindSPONGE] Step: 0, E_pot: 24502.51
[MindSPONGE] Step: 10, E_pot: 19268.096
[MindSPONGE] Step: 20, E_pot: 18033.639
[MindSPONGE] Step: 30, E_pot: 17534.414
[MindSPONGE] Step: 40, E_pot: 17256.162
[MindSPONGE] Step: 50, E_pot: 17058.648
[MindSPONGE] Step: 60, E_pot: 16888.668
[MindSPONGE] Step: 70, E_pot: 16721.688
[MindSPONGE] Step: 80, E_pot: 16543.371
[MindSPONGE] Step: 90, E_pot: 16343.916
[MindSPONGE] Step: 100, E_pot: 16117.062
[MindSPONGE] Step: 110, E_pot: 15860.616
[MindSPONGE] Step: 120, E_pot: 15576.784
[MindSPONGE] Step: 130, E_pot: 15271.274
[MindSPONGE] Step: 140, E_pot: 14951.434
[MindSPONGE] Step: 150, E_pot: 14624.309
[MindSPONGE] Step: 160, E_pot: 14295.655
[MindSPONGE] Step: 170, E_pot: 13969.825
[MindSPONGE] Step: 180, E_pot: 13650.449
[MindSPONGE] Step: 190, E_pot: 13340.956
[MindSPONGE] Step: 200, E_pot: 13044.489
[MindSPONGE] Step: 210, E_pot: 12763.801
[MindSPONGE] Step: 220, E_pot: 12500.975
[MindSPONGE] S

Step 3: NVT

In [5]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(system=mol,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',)
md.change_optimizer(opt)
cb_h5md = WriteH5MD(mol, '../p05_2.h5md', save_freq=10, write_image=False)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:38:35
[MindSPONGE] Step: 0, E_pot: 9545.592, E_kin: 2116.8677, E_tot: 11662.459, Temperature: 296.73737, Pressure: -5207.054, Volume: 7.5034766
[MindSPONGE] Step: 10, E_pot: 9883.159, E_kin: 1741.0917, E_tot: 11624.251, Temperature: 244.06201, Pressure: -5106.0312, Volume: 7.5034766
[MindSPONGE] Step: 20, E_pot: 9841.503, E_kin: 1760.5466, E_tot: 11602.05, Temperature: 246.78917, Pressure: -4400.4526, Volume: 7.5034766
[MindSPONGE] Step: 30, E_pot: 9733.656, E_kin: 1904.7642, E_tot: 11638.42, Temperature: 267.00522, Pressure: -4085.3672, Volume: 7.5034766
[MindSPONGE] Step: 40, E_pot: 9863.387, E_kin: 1800.3951, E_tot: 11663.782, Temperature: 252.37503, Pressure: -3983.5352, Volume: 7.5034766
[MindSPONGE] Step: 50, E_pot: 9855.344, E_kin: 1833.1392, E_tot: 11688.482, Temperature: 256.965, Pressure: -3443.8176, Volume: 7.5034766
[MindSPONGE] Step: 60, E_pot: 9804.4375, E_kin: 1905.1372, E_tot: 11709.574, Temperature: 267.05753, Pressure: 

Step 4: NPT

In [6]:
temp = 300
vgen = VelocityGenerator(temp)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(system=mol,
                time_step=1e-3,
                velocity=velocity,
                integrator='velocity_verlet',
                temperature=300,
                thermostat='langevin',
                pressure=1,
                barostat='berendsen',)
md.change_optimizer(opt)
cb_h5md = WriteH5MD(mol, '../p05_3.h5md', save_freq=10, write_image=False)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:39:09
[MindSPONGE] Step: 0, E_pot: 8899.774, E_kin: 2172.842, E_tot: 11072.616, Temperature: 304.58374, Pressure: -4309.3843, Volume: 7.5034766
[MindSPONGE] Step: 10, E_pot: 8813.215, E_kin: 2307.0513, E_tot: 11120.266, Temperature: 323.39685, Pressure: -3662.8848, Volume: 7.492286
[MindSPONGE] Step: 20, E_pot: 8742.744, E_kin: 2335.9219, E_tot: 11078.666, Temperature: 327.44385, Pressure: -3473.7102, Volume: 7.4837418
[MindSPONGE] Step: 30, E_pot: 8730.052, E_kin: 2275.0093, E_tot: 11005.061, Temperature: 318.90527, Pressure: -3626.5403, Volume: 7.4768953
[MindSPONGE] Step: 40, E_pot: 8729.175, E_kin: 2278.9512, E_tot: 11008.126, Temperature: 319.45786, Pressure: -2996.2168, Volume: 7.468244
[MindSPONGE] Step: 50, E_pot: 8767.596, E_kin: 2252.7585, E_tot: 11020.3545, Temperature: 315.78622, Pressure: -1699.9502, Volume: 7.46115
[MindSPONGE] Step: 60, E_pot: 8687.975, E_kin: 2283.0896, E_tot: 10971.064, Temperature: 320.03796, Pressure: 

Step 5: define CVs

In [7]:
phi = Torsion([3, 12, 13, 21])
psi = Torsion([12, 13, 21, 36])

Step 6: Normal MD

In [8]:
sim = WithEnergyCell(mol, energy)
vgen = VelocityGenerator(300)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(
    system=mol,
    integrator=VelocityVerlet(mol),
    thermostat=Langevin(mol, 300),
    time_step=1e-3,
    velocity=velocity,
    pressure=1,
    barostat='berendsen',
)
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})
cb_h5md = WriteH5MD(mol, '../p05_a.h5md', save_freq=100, write_image=False)
md.run(10000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:39:45
[MindSPONGE] Step: 0, E_pot: 8285.462, E_kin: 2152.406, E_tot: 10437.868, Temperature: 301.71906, Pressure: -4034.9805, Volume: 6.8978, phi: -0.91868556, psi: 1.8397787
[MindSPONGE] Step: 10, E_pot: 8326.53, E_kin: 2179.96, E_tot: 10506.49, Temperature: 305.5815, Pressure: -4167.9663, Volume: 6.8904366, phi: -0.9653797, psi: 1.8080351
[MindSPONGE] Step: 20, E_pot: 8192.573, E_kin: 2285.0464, E_tot: 10477.619, Temperature: 320.31226, Pressure: -3782.7473, Volume: 6.8839555, phi: -1.0855508, psi: 1.9250989
[MindSPONGE] Step: 30, E_pot: 8291.041, E_kin: 2196.0574, E_tot: 10487.099, Temperature: 307.83798, Pressure: -2524.3965, Volume: 6.877523, phi: -1.0858225, psi: 1.9805784
[MindSPONGE] Step: 40, E_pot: 8343.746, E_kin: 2107.7314, E_tot: 10451.478, Temperature: 295.4567, Pressure: -1506.3901, Volume: 6.873173, phi: -1.1144603, psi: 1.9747906
[MindSPONGE] Step: 50, E_pot: 8251.928, E_kin: 2223.326, E_tot: 10475.254, Temperature: 311.

Step 7: MetaD settings

In [9]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

Step 8: Enhanced Sampling using MetaD

In [10]:
sim = WithEnergyCell(mol, energy, bias=metad)
vgen = VelocityGenerator(300)
velocity = vgen(mol.shape, mol.atom_mass)
opt = UpdaterMD(
    system=mol,
    integrator=VelocityVerlet(mol),
    thermostat=Langevin(mol, 300),
    time_step=1e-3,
    velocity=velocity,
    pressure=1,
    barostat='berendsen',
)
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})
cb_h5md = WriteH5MD(mol, '../p05_b.h5md', save_freq=100, write_image=False)
md.run(10000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:44:23
[MindSPONGE] Step: 0, E_pot: 7275.9316, E_kin: 2174.1553, E_tot: 9450.087, Temperature: 304.76782, Pressure: 1247.0082, Volume: 5.4951367, phi: -1.159998, psi: 2.10881
[MindSPONGE] Step: 10, E_pot: 7284.276, E_kin: 2173.7817, E_tot: 9458.058, Temperature: 304.71545, Pressure: -186.46233, Volume: 5.4938927, phi: -1.1151779, psi: 2.0525212
[MindSPONGE] Step: 20, E_pot: 7335.8364, E_kin: 2112.9072, E_tot: 9448.744, Temperature: 296.18222, Pressure: 1648.7365, Volume: 5.4956145, phi: -0.8711893, psi: 1.9474769
[MindSPONGE] Step: 30, E_pot: 7401.7583, E_kin: 2045.3425, E_tot: 9447.101, Temperature: 286.71115, Pressure: 1074.1217, Volume: 5.5002975, phi: -0.9144371, psi: 2.0074143
[MindSPONGE] Step: 40, E_pot: 7360.918, E_kin: 2074.6594, E_tot: 9435.577, Temperature: 290.8207, Pressure: -754.69745, Volume: 5.501529, phi: -0.91778535, psi: 2.0099201
[MindSPONGE] Step: 50, E_pot: 7420.308, E_kin: 2043.0714, E_tot: 9463.38, Temperature: 286